In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_train = pd.read_csv('/kaggle/input/telecom-clients-prediction2/orange_small_churn_train_data.csv')
data_test = pd.read_csv('/kaggle/input/telecom-clients-prediction2/orange_small_churn_test_data.csv')
print(data_train.shape, data_test.shape)

In [ ]:
data_train.dropna(axis=0, subset=['labels'], how='any', inplace=True)
data_train.shape

In [ ]:
X_train = data_train.iloc[:,:-1]
y_train = data_train.iloc[:,-1:]

X_dropped = X_train.dropna(axis=1, thresh=9_000, how="any") 
cat_cols = X_dropped.select_dtypes(exclude=["number","bool_"]).columns
X_cat = X_dropped[cat_cols].astype('str').astype('category').apply(lambda x: x.cat.codes)

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
num_cols = list(set(X_dropped.columns) - set(cat_cols))
X_num = pd.DataFrame(scaler.fit_transform(X_dropped[num_cols]), columns=num_cols)
X_num.fillna(0, inplace=True)
X_num.apply(np.float32);

In [ ]:
X_new = X_dropped.astype('str')

In [ ]:
X_test = data_test[X_new.columns]
X_test_num = pd.DataFrame(scaler.transform(X_test[num_cols]), columns=num_cols)
X_test_cat = X_test[cat_cols].astype('str')
X_test_cat = X_test_cat.astype('category')
X_test_cat = X_test_cat.apply(lambda x: x.cat.codes)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)
X_test.shape

In [ ]:
X_test = data_test[X_new.columns].astype('str')

In [ ]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV

grid = RandomizedSearchCV(estimator=CatBoostClassifier(verbose=False, allow_writing_files=False,
                                                       cat_features=X_new.columns), 
                          param_distributions={
                                'n_estimators': range(100, 600, 200),
                                'max_depth': range(2, 5),
                                'learning_rate': np.arange(0.1, 1, 0.2)
                          }, 
                          scoring='roc_auc')

In [ ]:
%%time
grid.fit(X_new, y_train)

In [ ]:
preds = grid.predict_proba(X_test)[:,1:]
ans = pd.DataFrame(preds, columns=['result'])
ans.to_csv('sample_sub.csv', header=['result'], index_label='Id')
ans